# First example of a convNet

With this example we will understand how to build a convolution neural network and apply it to hand-written digit classification.

Import the libraries we will need to define the DL network and load the MNIST set of data

In [ ]:
from keras import models
from keras import layers
from keras.datasets import mnist
import matplotlib.pyplot as plt
from keras.utils import to_categorical

(train_images_all, train_labels), (test_images, test_labels) = mnist.load_data()

Change the shape of the images from a 2D array to a tensor. The labels are also changed from digit to
categorical. Finally the data are normalized and a validation set is defined.

In [ ]:
train_labels_all = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

train_images = train_images_all[0:20000]
validation_images = train_images_all[50000:60000]
train_labels = train_labels_all[0:20000]
validation_labels = train_labels_all[50000:60000]

train_images = train_images.reshape((20000,28,28,1))
train_images = train_images.astype('float32')/255

validation_images = validation_images.reshape((10000,28,28,1))
validation_images = validation_images.astype('float32')/255

test_images = test_images.reshape((10000,28,28,1))
test_images = test_images.astype('float32')/255

As an example, plot the nth image of the training image

In [ ]:
n = 2
digit = train_images_all[n]
plt.imshow(digit, cmap=plt.cm.binary)
plt.show()

label = train_labels[n]
print('The associated label for this image is ', label)

Definition of the convNet applied to digit classification

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(8, (3, 3), activation='relu', input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(16,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(32,(3,3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

Compile and train the model

In [ ]:
model.compile(optimizer='rmsprop', 
                loss='categorical_crossentropy',
                metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=5, batch_size=64,
                   validation_data = (validation_images, validation_labels))

Calculate the accuracy of the model using the test set

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('test_acc:', test_acc)

Below the accuracy for the training and validation sets are plotted

In [ ]:
history_dict = history.history
print( history_dict.keys() )

acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

n = len(acc_values)
epochs = range(1, n+1)

plt.plot(epochs, acc_values, 'bo', label='Training acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

Show what and how the network is learning

In [ ]:
layer_outputs = [layer.output for layer in model.layers[:6]]
activation_model = models.Model(inputs = model.input, outputs = layer_outputs)

activations = activation_model.predict(test_images)
first_layer_activation = activations[1]
print(first_layer_activation.shape)
            

In [ ]:
import numpy as np

layer_names = []

for layer in model.layers[:3]:
    layer_names.append(layer.name)
    
images_per_row = 8

for layer_name, layer_activation in zip(layer_names, activations):
    n_features = layer_activation.shape[-1]
    
    size = layer_activation.shape[1]
    
    n_cols = n_features // images_per_row
    display_grid = np.zeros((size * n_cols, images_per_row * size))
    
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[1,:,:,col * images_per_row + row]
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col+1) * size,
                        row * size : (row+1) * size] = channel_image
            
scale = 1. / size
plt.figure(figsize=(scale * display_grid.shape[1],
                   scale * display_grid.shape[0]))
plt.title(layer_name)
plt.grid(False)
plt.imshow(display_grid, aspect='auto', cmap='viridis')